Many researchers work on developing fair classifiers. In this notebook we will train some of these methods and analyze how they perform. 

How might you try to make the classifier we built last time more fair? Would you try to change the data or maybe adjust the predictions?

# Fairness Methods
 
 Methods for achieving algorithmic fairness fall into one of three categories:
 - Pre-processing
 - In-processing
 - Post-processing
 
 ## Pre-processing
 
 Pre-processing methods change the data in order to encourage the algorithm to be more fair. For instance we saw that African-Americans had a higher base rate of recidivism than Caucasians. Maybe if we changed the data so that the base rates looked equal (by for instance reweighing the training data), then the algorithm would learn a more fair classification. 
 
 Pre-processing is method agnostic, but it makes no guarantee about how fair the resulting algorithm will be.
 
 ## In-processing
 
 In-processing methods change the actual learning procedure for the model. Most models are learned via an optimization; in-processing methods often constrain that optimization to a *fair* subspace.
 
 In-processing methods are not method agnostic, so they do not generalize well, but you can get fairness guarantees.
 
 ## Post-processing
 
 Post-processing methods take the predictions of any algorithm and adjust them to be more fair. For instance if our algorithm releases 50% of Caucasians but only 30% of African-Americans, then a most processing method could try to achieve demographic parity by randomly releasing 30% of African-Americans who otherwise would have been detained. How would this achieve demographic parity? 

**Your answer**:

As usual we will import pandas and numpy. We also need to import an optimization package cvxpy. And we will import the fair methods from folders in this directory.

In [4]:
import pandas as pd
import numpy as np
import cvxpy as cvx
from equalized_odds_and_calibration.eq_odds import Model

In [5]:
train = np.loadtxt("train.csv")
test = np.loadtxt("test.csv")
X_pandas = pd.read_csv("X_pandas.csv")
feat_map = list(X_pandas.columns)[1:]
feat_map.append("label")
feat_map.append("prediction")

# Post-processing for Equalized Odds

We will explore a method developed by Moritz Hardt et al (https://arxiv.org/pdf/1610.02413.pdf) that uses post-processing techniques to achieve equalized odds. 

Their methods require a Model data structure that takes two inputs: 1) a matrix of the algorithm predictions and 2) a matrix of the true labels. We have to create such a Model object for each demographic group.

In [11]:
feat_map

['priors_count',
 'juv_fel_count',
 'juv_misd_count',
 'juv_other_count',
 'age',
 'c_charge_degree_F',
 'c_charge_degree_M',
 'race_African-American',
 'race_Caucasian',
 'race_Hispanic',
 'race_Other',
 'sex_Female',
 'sex_Male',
 'label',
 'prediction']

In [20]:
train.shape

(3703, 15)

In [22]:
afr_am_train = train[train[:,feat_map.index("race_African-American")] == 1]
cauc_train = train[train[:,feat_map.index("race_Caucasian")] == 1]
afr_am_test = test[test[:,feat_map.index("race_African-American")] == 1]
cauc_test = test[test[:,feat_map.index("race_Caucasian")] == 1]

In [10]:
feat_map.index("prediction")

14

In [36]:
model_afr_am_train = Model(afr_am_train[:, feat_map.index("prediction")], afr_am_train[:, feat_map.index("label")])
model_cauc_train = Model(cauc_train[:, feat_map.index("prediction")], cauc_train[:, feat_map.index("label")])
model_afr_am_test = Model(afr_am_test[:, feat_map.index("prediction")], afr_am_test[:, feat_map.index("label")])
model_cauc_test = Model(cauc_test[:, feat_map.index("prediction")], cauc_test[:, feat_map.index("label")])

To check that we have done this properly, we can print the models. This shows us the accuracy, FPR, FNR, base rate and mean prediction.

In [25]:
model_afr_am_train

Accuracy:	0.685
F.P. cost:	0.309
F.N. cost:	0.321
Base rate:	0.518
Avg. score:	0.501

In [26]:
model_cauc_train

Accuracy:	0.665
F.P. cost:	0.152
F.N. cost:	0.610
Base rate:	0.400
Avg. score:	0.247

Now let's call the method for post-processing. This method gives us mixing rates that tell us how to randomly resample predicted labels to equalize false positive rates and false negative rates

In [31]:
_, _, mixing_rates = Model.eq_odds(model_afr_am_train, model_cauc_train)

Now we can correct the predictions using these mixing rates

In [32]:
model_afr_am_train_corrected, model_cauc_train_corrected = Model.eq_odds(model_afr_am_train, model_cauc_train, mixing_rates)

In [33]:
model_afr_am_train_corrected

Accuracy:	0.612
F.P. cost:	0.185
F.N. cost:	0.577
Base rate:	0.518
Avg. score:	0.308

In [34]:
model_cauc_train_corrected

Accuracy:	0.649
F.P. cost:	0.193
F.N. cost:	0.587
Base rate:	0.400
Avg. score:	0.281

Did this achieve equalized odds? 

**Your answer**

We should evaluate how the mixing performs on the test partition. Why do we need to do this? Should we get new mixing rates on the test partition or use the mixing rates learned from the train partition?

**Your answer**

In [37]:
model_afr_am_test_corrected, model_cauc_test_corrected = Model.eq_odds(model_afr_am_test, model_cauc_test, mixing_rates)

In [38]:
model_cauc_test_corrected

Accuracy:	0.672
F.P. cost:	0.178
F.N. cost:	0.576
Base rate:	0.378
Avg. score:	0.271

In [39]:
model_afr_am_test_corrected

Accuracy:	0.592
F.P. cost:	0.207
F.N. cost:	0.585
Base rate:	0.531
Avg. score:	0.318

In [40]:
model_cauc_test

Accuracy:	0.691
F.P. cost:	0.136
F.N. cost:	0.594
Base rate:	0.378
Avg. score:	0.238

In [41]:
model_afr_am_test

Accuracy:	0.688
F.P. cost:	0.316
F.N. cost:	0.307
Base rate:	0.531
Avg. score:	0.516

What notion of fairness does this method achieve, if any? What is the effect of this method on the accuracy? Do you think we should use this method for our pre-trial bail application?

# Fair Reductions

Now we will consider a different fairness algorithm developed by 

In [1]:
import fairlearn.moments as moments
import fairlearn.classred as red
from sklearn.linear_model import LogisticRegression

Their method will 

In [39]:
logreg = LogisticRegression(C=100000, max_iter=10000, solver = "lbfgs")

In [65]:
X = pd.DataFrame(train[:, np.r_[0:7, 11:13]])#pd.DataFrame(train[:, :-2])
A = pd.Series(train[:,feat_map.index('race_African-American')])
Y = pd.Series(train[:,-2])

We call the method expgrad (which stands for exponentiated gradient) to learn the fair algorithm. We specify cons=moments.EO() for equalized odds (or if we wanted demographic partity then we would specify moments.DP()). eps is the allowable error tolerance in the fairness goal. As we decrease eps toward 0, we should get closer to the fairness target. As we increase eps away from 0, we would expect to have a more accurate classifier that is farther from the fairness target.

In [137]:
res_tuple = red.expgrad(dataX = X, dataA = A, dataY = Y, learner=logreg, cons=moments.EO(), eps=0.001)

In [70]:
res = res_tuple._asdict()
Q = res["best_classifier"]

Q is the best classifier learned by this method. We can all Q(dat) where dat is a pandas dataframe containing our covariates. This will return a pandas Series with the predicted probabilities. 

Does this achieve equalized odds?

In [145]:
tpr_afr_am = Q(X)[np.logical_and(A==1, Y==1)].round().mean()
tpr_cauc = Q(X)[np.logical_and(A==0, Y==1)].round().mean()
fpr_afr_am = Q(X)[np.logical_and(A==1, Y==0)].round().mean()
fpr_cauc = Q(X)[np.logical_and(A==0, Y==0)].round().mean()
print("African-American tpr: {0:.5f}. Caucasian tpr: {1:.5f}".format(tpr_afr_am, tpr_cauc))
print("African-American fpr: {0:.5f}. Caucasian fpr: {1:.5f}".format(fpr_afr_am, fpr_cauc))

African-American tpr: 0.19115. Caucasian tpr: 0.23286
African-American fpr: 0.19006. Caucasian fpr: 0.33333


In [146]:
(Q(X).round() != Y).mean()

0.5071563597083446

In [147]:
(Q(X) - Y).abs().mean()

0.4487359537976658

There is a slight problem with the calculations we just did. What is the problem? *Hint* can you use X_test (created in the next line) to solve this problem? 

In [58]:
X_test = pd.DataFrame(test[:, :-2])

In [55]:
train[1, feat_map.index('sex_Female')]

0.0

In [144]:
error = moments.MisclassError()
error.init(X, A, Y)
error.gamma(Q)[0]

0.4487359537976658

In [136]:
print(Q(X)[A==1].mean())
print(Q(X)[A==0].mean())
print(Q(X).mean())

print(Q(X)[A==1].round().mean())
print(Q(X)[A==0].round().mean())
print(Q(X).round().mean())

0.2784825480993945
0.27655390253500134
0.2775539023955563
0.190625
0.2938867077958497
0.24034566567647853


In [62]:
train[100, np.r_[0:7, 11:13]]

array([28.,  0.,  0.,  0., 57.,  1.,  0.,  0.,  1.])

In [65]:
train[10, -2]

0.0

In [96]:
attrs = [str(x) for x in 'AAAAAAA' 'BBBBBBB']
labls = [int(x) for x in '0110100' '0010111']
feat1 = [int(x) for x in '0110101' '0111101']
feat2 = [int(x) for x in '0000100' '0000011']
feat3 = [int(x) for x in '1111111' '1111111']

dataX = pd.DataFrame({"feat1": feat1, "feat2": feat2, "feat3": feat3})
dataY = pd.Series(labls)
dataA = pd.Series(attrs)

In [113]:
res_tuple2 = red.expgrad(dataX, dataA, dataY, learner=logreg, cons=moments.DP(), eps=0.001)

In [98]:
res2 = res_tuple2._asdict()
q = res_tuple2._asdict()['best_classifier']

In [109]:
q(dataX)[dataA == 'A'].mean()-q(dataX)[dataA == 'B'].mean()

-0.00199999956032737

In [108]:
q(dataX)[dataA == 'B'].mean()

0.502499999451681

In [81]:
0.0009999997801637406

0.5

In [82]:
q(dataX)[dataA == 'B']

7     2.131304e-01
8     4.402609e-01
9     4.402609e-01
10    4.402609e-01
11    4.402609e-01
12    1.098689e-08
13    7.868696e-01
dtype: float64

In [105]:
q(dataX)

0     0.4965
1     0.5035
2     0.5035
3     0.4965
4     0.5035
5     0.4965
6     0.5035
7     0.4965
8     0.5035
9     0.5035
10    0.5035
11    0.5035
12    0.5035
13    0.5035
dtype: float64

In [106]:
q(dataX).round()

0     0.0
1     1.0
2     1.0
3     0.0
4     1.0
5     0.0
6     1.0
7     0.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
dtype: float64

In [101]:
disp = moments.DP()
disp.init(dataX, dataA, dataY)
res2["disp"] = disp.gamma(q).max()

In [88]:
dataX.shape

(20, 3)

In [102]:
res2

OrderedDict([('best_classifier',
              <function fairlearn.classred.expgrad.<locals>.<lambda>(X)>),
             ('best_gap', 4.396726316358013e-10),
             ('classifiers',
              0    LogisticRegression(C=100000, class_weight=None...
              1    LogisticRegression(C=100000, class_weight=None...
              dtype: object),
             ('weights', 0    0.5035
              1    0.4965
              dtype: float64),
             ('last_t', 5),
             ('best_t', 5),
             ('n_oracle_calls', 17),
             ('disp', 0.0009999997801637406)])

In [112]:
res_tuple3 = red.expgrad(dataX, dataA, dataY, learner=logreg, cons=moments.EO(), eps=0.01)

In [114]:
res3 = res_tuple3._asdict()
q3 = res3['best_classifier']

In [123]:
q3(dataX)[np.logical_and(dataA=='A',dataY == 1)].round().mean()

0.3333333333333333

In [124]:
q3(dataX)[np.logical_and(dataA=='B',dataY == 1)].round().mean()

0.5

In [125]:
q3(dataX)[np.logical_and(dataA=='A',dataY == 1)].mean()

0.5720000029208396

In [126]:
q3(dataX)[np.logical_and(dataA=='B',dataY == 1)].mean()

0.5895000016750881

In [129]:
q3(dataX)[dataY == 1].mean()- q3(dataX)[np.logical_and(dataA=='B',dataY == 1)].mean()

-0.007499999466106577

In [130]:
q3(dataX)[dataY == 1].mean()- q3(dataX)[np.logical_and(dataA=='A',dataY == 1)].mean()

0.009999999288141881

In [134]:
q3(dataX)[dataY == 1].round().mean()- q3(dataX)[np.logical_and(dataA=='B',dataY == 1)].round().mean()

-0.07142857142857145

In [132]:
disp3 = moments.DP()
disp3.init(dataX, dataA, dataY)
res3["disp"] = disp3.gamma(q3).max()

In [133]:
res3

OrderedDict([('best_classifier',
              <function fairlearn.classred.expgrad.<locals>.<lambda>(X)>),
             ('best_gap', 1.1009944067552624e-09),
             ('classifiers',
              0    LogisticRegression(C=100000, class_weight=None...
              1    LogisticRegression(C=100000, class_weight=None...
              2    LogisticRegression(C=100000, class_weight=None...
              3    LogisticRegression(C=100000, class_weight=None...
              4    LogisticRegression(C=100000, class_weight=None...
              5    LogisticRegression(C=100000, class_weight=None...
              dtype: object),
             ('weights', 0    4.907486e-09
              1    3.160000e-01
              2    3.260000e-01
              3    1.806719e-09
              4    3.580000e-01
              5    1.368589e-09
              dtype: float64),
             ('last_t', 5),
             ('best_t', 5),
             ('n_oracle_calls', 23),
             ('disp', 0.02628571360095355